In [ ]:
from dotenv import load_dotenv
from llama_index.llms.openai import OpenAI
from langchain.embeddings import HuggingFaceEmbeddings
from llama_index.core.settings import Settings
from llama_index.core import (
    load_index_from_storage,
    StorageContext,
)
from llama_index.core.indices.vector_store import VectorStoreIndex
from llama_index.embeddings.langchain import LangchainEmbedding
from llama_index.vector_stores.faiss import FaissVectorStore

In [ ]:
load_dotenv("../../.env")

In [ ]:
embed_model_name = "intfloat/multilingual-e5-small"
embed_model = LangchainEmbedding(HuggingFaceEmbeddings(model_name=embed_model_name))
Settings.embed_model = embed_model
Settings.llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

In [ ]:
# load index from disk
vector_store = FaissVectorStore.from_persist_dir("../data/llama_faiss")
storage_context = StorageContext.from_defaults(vector_store=vector_store, persist_dir="../data/llama_faiss")
index: VectorStoreIndex = load_index_from_storage(storage_context=storage_context)

In [ ]:
type(index)

In [ ]:
query_engine = index.as_query_engine(similarity_top_k=5)
# response = query_engine.query("DX白書の重要なポイントをMECEに教えてくれませんか？")
response = query_engine.query("DXにおける重要な課題をMECEに教えてくれませんか？")

In [ ]:
response

In [ ]:
print(response.response)

In [ ]:
len(response.source_nodes)

In [ ]:
response.source_nodes[0].node.text

In [ ]:
for idx, sn in enumerate(response.source_nodes):
    print("-" * 80)
    print(f"[情報源{idx+1:02d}]")
    print("-" * 10)
    print(sn.node.text)
    meta = sn.node.metadata
    # print(f"[{meta['file_path']}:{meta['file_name']}:{meta['page_label']}]")
    print(f"[{meta['file']}:{meta['page']}]")
    # print(sn.metadata)

# with LangChain

In [ ]:
import langchain
import langchain.agents
from langchain_openai import ChatOpenAI
from langchain.agents import Tool, initialize_agent, load_tools
from langchain.chains.conversation.memory import ConversationBufferMemory

In [ ]:
def create_llama_index_tool(similarity_top_k: int = 5):
    return Tool(
        name="DX白書検索",
        func=lambda q: str(index.as_query_engine(similarity_top_k=5).query(q)),
        description="DXに関する情報を検索したり見つけるときに有効です。",
        # description="Useful for search and find the information in the DX白書",
        return_direct=False,
    )

In [ ]:
model_name = "gpt-3.5-turbo"
llm = ChatOpenAI(temperature=0.1, model_name=model_name)

tools = load_tools(["google-search", "llm-math", "wikipedia"], llm=llm)  # , "terminal"
tools += [create_llama_index_tool(similarity_top_k=5)]

In [ ]:
agent = initialize_agent(
    tools,
    # llm=OpenAI(temperature=0),
    llm=llm,
    agent=langchain.agents.AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

In [ ]:
agent.run(input="DX白書の重要なポイントをMECEに教えてくれませんか？")